In [ ]:
initial  dy


Service detection source confirmation

For RDM Service write-back, can we confirm:

Which exact Silver source tables should be used to detect new service values?

WIP → (which column?)

DRJ → (which column?)

In [ ]:
Service detection source confirmation

For RDM Service write-back, can we confirm:

Which exact Silver source tables should be used to detect new service values?

WIP → (which column?)

DRJ → (which column?)
 


Should we follow the same pattern as other RDM entities (trim/lower + anti-join against silver_rdm_service)?

2️⃣ Schema confirmation for _add table

Should silver_rdm_service_add mirror the exact schema of silver_rdm_service (excluding governance columns if not required)?

Any additional fields expected specifically for Service (e.g., master_service, business_unit)?

3️⃣ Write-back behaviour

Should Service write-back follow the same nightly Power Automate pattern as other RDM lists?

Any additional validation or approval rule specific to Service?

4️⃣ Lakehouse / Semantic model handling

Confirm that _add table should be:

Added as shortcut in silver_rdm lakehouse

Exposed in existing RDM semantic model (same pattern as others)

In [ ]:
code

In [ ]:
join key = service_src_name + z_src_system_id (instance ignore)
-- =========================================
-- RDM SERVICE ADD (Pattern A: system-level)
-- Join on: service_src_name + z_src_system_id
-- =========================================

DROP TABLE IF EXISTS silver_rdm_service_add;

CREATE TABLE silver_rdm_service_add AS
WITH AllServices AS (

    SELECT DISTINCT
        lower(trim(cprod_service)) AS service_src_name,
        z_src_system_id
    FROM silver_care_product
    WHERE cprod_service IS NOT NULL
      AND trim(cprod_service) <> ''
      AND lower(trim(cprod_service)) <> 'unknown'

)
SELECT
    a.z_src_system_id,
    a.service_src_name
FROM AllServices a
LEFT JOIN silver_rdm_service r
    ON lower(trim(r.service_src_name)) = a.service_src_name
   AND r.z_src_system_id = a.z_src_system_id
WHERE r.service_src_name IS NULL;

--Join key = service_src_name + z_src_system_id + service_src_sys_inst_id (instance include)

In [ ]:
-- =========================================
-- RDM SERVICE ADD (Pattern B: instance-aware)
-- Join on: service_src_name + z_src_system_id + service_src_sys_inst_id
-- =========================================

DROP TABLE IF EXISTS silver_rdm_service_add;

CREATE TABLE silver_rdm_service_add AS
WITH AllServices AS (

    SELECT DISTINCT
        lower(trim(cprod_service)) AS service_src_name,
        z_src_system_id,
        z_src_system_instance      AS service_src_sys_inst_id
    FROM silver_care_product
    WHERE cprod_service IS NOT NULL
      AND trim(cprod_service) <> ''
      AND lower(trim(cprod_service)) <> 'unknown'

)
SELECT
    a.z_src_system_id,
    a.service_src_sys_inst_id,
    a.service_src_name
FROM AllServices a
LEFT JOIN silver_rdm_service r
    ON lower(trim(r.service_src_name))   = a.service_src_name
   AND r.z_src_system_id                 = a.z_src_system_id
   AND r.service_src_sys_inst_id         = a.service_src_sys_inst_id
WHERE r.service_id IS NULL;

In [ ]:
SELECT * FROM silver_rdm_service_add LIMIT 50;

SELECT count(*) FROM silver_rdm_service_add;

last 

In [ ]:
Version A – match on service_src_name + z_src_system_id + service_src_sys_inst_id

existed pattern _add blocks  (system-level + instance-level).

-- Version A: by source name + system id + source system instance id
DROP TABLE IF EXISTS silver_rdm_service_add_vA;

CREATE OR REPLACE TABLE silver_rdm_service_add_vA AS
SELECT DISTINCT
    cp.z_src_system_id,
    lower(trim(cp.cprod_service)) AS service_src_name,
    cp.z_src_system_instance     AS service_src_sys_inst_id
FROM silver_care_product cp
LEFT JOIN silver_rdm_service r
    ON lower(trim(r.service_src_name)) = lower(trim(cp.cprod_service))
   AND r.z_src_system_id = cp.z_src_system_id
   AND r.service_src_sys_inst_id = cp.z_src_system_instance
WHERE r.service_src_name IS NULL
  AND cp.cprod_service IS NOT NULL
  AND trim(cp.cprod_service) <> '';

In [ ]:
-- Version B: by service_src_id + system id + source system instance id
DROP TABLE IF EXISTS silver_rdm_service_add_vB;

CREATE OR REPLACE TABLE silver_rdm_service_add_vB AS
SELECT DISTINCT
    cp.z_src_system_id,
    cp.<PUT_SERVICE_SRC_ID_COLUMN_HERE> AS service_src_id,
    cp.z_src_system_instance            AS service_src_sys_inst_id
FROM silver_care_product cp
LEFT JOIN silver_rdm_service r
    ON r.service_src_id = cp.<PUT_SERVICE_SRC_ID_COLUMN_HERE>
   AND r.z_src_system_id = cp.z_src_system_id
   AND r.service_src_sys_inst_id = cp.z_src_system_instance
WHERE r.service_src_id IS NULL
  AND cp.<PUT_SERVICE_SRC_ID_COLUMN_HERE> IS NOT NULL;

In [ ]:
SELECT 'vA' AS ver, count(*) cnt FROM silver_rdm_service_add_vA
UNION ALL
SELECT 'vB' AS ver, count(*) cnt FROM silver_rdm_service_add_vB;

-- rows in vA not in vB
SELECT * FROM silver_rdm_service_add_vA
EXCEPT
SELECT z_src_system_id, service_src_name, service_src_sys_inst_id
FROM silver_rdm_service_add_vB;  -- (हे तेव्हाच चालेल जेव्हा vB मध्ये service_src_name असेल. नसेल तर adjust)

-- rows in vB not in vA
SELECT * FROM silver_rdm_service_add_vB
EXCEPT
SELECT z_src_system_id, NULL AS service_src_id, service_src_sys_inst_id
FROM silver_rdm_service_add_vA;  -- adjust as per columns

start


In [ ]:
%%sql
--rdm instance-level uniqueness check
SELECT service_src_sys_inst_id, COUNT(*) AS rdm_rows
FROM silver_rdm_service
GROUP BY service_src_sys_inst_id
ORDER BY service_src_sys_inst_id;

In [ ]:
%%sql
--care product instance-level uniqueness check
SELECT z_src_system_instance, COUNT(*) AS cp_rows
FROM silver_care_product
GROUP BY z_src_system_instance
ORDER BY z_src_system_instance;

In [ ]:
%%sql
-- care product- service name service candidaete check(per instane)
SELECT
  z_src_system_instance,
  lower(trim(cprod_service)) AS cprod_service_norm,
  COUNT(*) AS rows_cnt
FROM silver_care_product
WHERE cprod_service IS NOT NULL
  AND trim(cprod_service) <> ''
GROUP BY z_src_system_instance, lower(trim(cprod_service))
ORDER BY z_src_system_instance, rows_cnt DESC;

In [ ]:
%%sql
-- rdm service - service name candidate check (per instance)
SELECT
  service_src_sys_inst_id,
  lower(trim(service_src_name)) AS service_src_name_norm,
  COUNT(*) AS rows_cnt
FROM silver_rdm_service
WHERE service_src_name IS NOT NULL
  AND trim(service_src_name) <> ''
GROUP BY service_src_sys_inst_id, lower(trim(service_src_name))
ORDER BY service_src_sys_inst_id, rows_cnt DESC;

111111111111111111111111111111111111111111111111111111111111

In [ ]:
%%sql
--only instance based add(pattern c)
CREATE OR REPLACE TABLE silver_rdm_service_add_v1 AS

SELECT DISTINCT
    cp.z_src_system_instance AS service_src_sys_inst_id,
    'unknown'                AS service_src_name,
    cp.z_src_system_id       AS z_src_system_id
FROM silver_care_product cp

LEFT JOIN silver_rdm_service r
    ON r.service_src_sys_inst_id = cp.z_src_system_instance

WHERE r.service_src_sys_inst_id IS NULL;

In [ ]:
SELECT * FROM silver_rdm_service_add_v1;

In [ ]:
SELECT COUNT(*) FROM silver_rdm_service_add_v1;

In [ ]:
%%sql

SELECT DISTINCT
    z_src_system_id,
    lower(trim(cprod_service)) AS service_name
FROM silver_care_product
WHERE cprod_service IS NOT NULL
  AND trim(cprod_service) <> '';

In [ ]:
%%sql

SELECT DISTINCT
    z_src_system_id,
    lower(trim(cprod_service)) AS service_name
FROM silver_care_product
WHERE cprod_service IS NOT NULL
  AND trim(cprod_service) <> '';

In [ ]:
%%sql

CREATE OR REPLACE TABLE silver_rdm_service_add_v2 AS

SELECT DISTINCT
    cp.z_src_system_id,
    lower(trim(cp.cprod_service)) AS service_src_name
FROM silver_care_product cp

LEFT JOIN silver_rdm_service r
    ON r.z_src_system_id = cp.z_src_system_id
   AND lower(trim(r.service_src_name)) = lower(trim(cp.cprod_service))

WHERE cp.cprod_service IS NOT NULL
  AND trim(cp.cprod_service) <> ''
  AND r.service_src_name IS NULL;

In [ ]:
hi


In [ ]:
%%sql

CREATE OR REPLACE TABLE silver_rdm_service_add_final AS

SELECT DISTINCT
    cp.z_src_system_id,
    lower(trim(cp.cprod_service)) AS service_src_name
FROM silver_care_product cp

LEFT JOIN silver_rdm_service r
    ON r.z_src_system_id = cp.z_src_system_id
   AND lower(trim(r.service_src_name)) = lower(trim(cp.cprod_service))

WHERE cp.cprod_service IS NOT NULL
  AND trim(cp.cprod_service) <> ''
  AND r.service_src_name IS NULL;